IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M2\NO_FAULT2


VOLO m2 FAULT 10%

In [3]:
rcou_m2_nofault = pd.read_csv("RCOU.csv")
rcou_m2_nofault = rcou_m2_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)
rcou_m2_nofault = rcou_m2_nofault[((rcou_m2_nofault['C9'] >= 1350) & (rcou_m2_nofault['C10'] >= 1350) & (rcou_m2_nofault['C11'] >= 1350) & (rcou_m2_nofault['C12'] >= 1350) & (rcou_m2_nofault['C13'] >= 1350) & (rcou_m2_nofault['C14']>= 1350))]
display(rcou_m2_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
5402,163535671,1390,1407,1412,1385,1400,1398
5403,163546021,1382,1426,1447,1359,1421,1387
5404,163556077,1378,1442,1450,1368,1414,1407
5405,163566849,1428,1405,1402,1431,1408,1425
5406,163577867,1452,1391,1393,1450,1419,1426
...,...,...,...,...,...,...,...
12788,244588431,1556,1418,1490,1492,1444,1535
12789,244598703,1559,1408,1492,1484,1453,1521
12790,244609809,1577,1376,1489,1481,1476,1493
12791,244620798,1575,1371,1518,1443,1508,1454


In [4]:
xkf1_m2_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m2_nofault = xkf1_m2_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m2_nofault = xkf1_m2_nofault[((xkf1_m2_nofault['TimeUS'] >= 163535671) & (xkf1_m2_nofault['TimeUS'] <= 244631223	))]
xkf1_m2_nofault = xkf1_m2_nofault.reset_index(drop=True)
print(xkf1_m2_nofault)

         TimeUS  Roll  Pitch     Yaw
0     163543665  1.17  -0.58   13.28
1     163554261  1.14  -0.58   13.28
2     163564285  1.19  -0.56   13.29
3     163575443  1.23  -0.54   13.30
4     163586483  1.30  -0.52   13.28
...         ...   ...    ...     ...
7385  244586271 -4.88  -0.88  358.90
7386  244596618 -4.80  -0.96  358.86
7387  244607605 -4.77  -1.02  358.82
7388  244618752 -4.71  -1.02  358.80
7389  244629015 -4.67  -1.04  358.77

[7390 rows x 4 columns]


In [5]:
att_m2_nofault = pd.read_csv("ATT.csv")
att_m2_nofault = att_m2_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m2_nofault = att_m2_nofault[((att_m2_nofault['TimeUS'] >= 163535671) & (att_m2_nofault['TimeUS'] <= 244631223	))]
att_m2_nofault=att_m2_nofault.reset_index(drop=True)
print(att_m2_nofault)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0     163545292    -0.13  1.17     -0.07  -0.58   13.14   13.28
1     163555319    -0.15  1.14     -0.06  -0.58   13.14   13.28
2     163566291    -0.16  1.19     -0.05  -0.56   13.14   13.29
3     163577191    -0.18  1.23     -0.05  -0.54   13.14   13.30
4     163588078    -0.19  1.30     -0.04  -0.52   13.14   13.28
...         ...      ...   ...       ...    ...     ...     ...
7386  244587670    -2.61 -4.88     -0.90  -0.88    0.26  358.91
7387  244597835    -2.50 -4.80     -0.89  -0.96    0.26  358.87
7388  244609128    -2.42 -4.77     -0.87  -1.02    0.26  358.83
7389  244620166    -2.34 -4.71     -0.86  -1.02    0.26  358.81
7390  244630400    -2.23 -4.67     -0.85  -1.04    0.26  358.78

[7391 rows x 7 columns]


In [6]:
if len(att_m2_nofault)>len(xkf1_m2_nofault):
    to_add=att_m2_nofault[len(xkf1_m2_nofault):]
    att_m2_nofault=att_m2_nofault[:len(xkf1_m2_nofault)]

if len(xkf1_m2_nofault)>len(att_m2_nofault):
    to_add=xkf1_m2_nofault[len(att_m2_nofault):]

for idx,i in enumerate(att_m2_nofault['Roll']):
    if(xkf1_m2_nofault['Roll'][idx] != i):
        att_m2_nofault['Roll'][idx] = ((att_m2_nofault['Roll'][idx] + xkf1_m2_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m2_nofault['Pitch']):
    if(xkf1_m2_nofault['Pitch'][idx] != i):
        att_m2_nofault['Pitch'][idx] = ((att_m2_nofault['Pitch'][idx] + xkf1_m2_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m2_nofault['Yaw']):
    if(xkf1_m2_nofault['Yaw'][idx] != i):
        att_m2_nofault['Yaw'][idx] = ((att_m2_nofault['Yaw'][idx] + xkf1_m2_nofault['Yaw'][idx])/2)

att_m2_nofault=pd.concat([att_m2_nofault,to_add])

print(att_m2_nofault)

C:\Users\chiar\AppData\Local\Temp\ipykernel_23420\600786250.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m2_nofault['Roll'][idx] = ((att_m2_nofault['Roll'][idx] + xkf1_m2_nofault['Roll'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_23420\600786250.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m2_nofault['Pitch'][idx] = ((att_m2_nofault['Pitch'][idx] + xkf1_m2_nofault['Pitch'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_23420\600786250.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

         TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw      Yaw
0     163545292    -0.13  1.170     -0.07  -0.58   13.14   13.280
1     163555319    -0.15  1.140     -0.06  -0.58   13.14   13.280
2     163566291    -0.16  1.190     -0.05  -0.56   13.14   13.290
3     163577191    -0.18  1.230     -0.05  -0.54   13.14   13.300
4     163588078    -0.19  1.300     -0.04  -0.52   13.14   13.280
...         ...      ...    ...       ...    ...     ...      ...
7386  244587670    -2.61 -4.840     -0.90  -0.92    0.26  358.885
7387  244597835    -2.50 -4.785     -0.89  -0.99    0.26  358.845
7388  244609128    -2.42 -4.740     -0.87  -1.02    0.26  358.815
7389  244620166    -2.34 -4.690     -0.86  -1.03    0.26  358.790
7390  244630400    -2.23 -4.670     -0.85  -1.04    0.26  358.780

[7391 rows x 7 columns]


In [7]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m2_nofault = pd.read_csv("ESC_0.csv")
esc_0_m2_nofault = esc_0_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m2_nofault = esc_0_m2_nofault[((esc_0_m2_nofault['TimeUS'] >= 163535671) & (esc_0_m2_nofault['TimeUS'] <= 244631223))]
esc_0_m2_nofault=esc_0_m2_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m2_nofault = pd.read_csv("ESC_1.csv")
esc_1_m2_nofault = esc_1_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m2_nofault = esc_1_m2_nofault[((esc_1_m2_nofault['TimeUS'] >= 163535671) & (esc_1_m2_nofault['TimeUS'] <= 244631223))]
esc_1_m2_nofault=esc_1_m2_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m2_nofault = pd.read_csv("ESC_2.csv")
esc_2_m2_nofault = esc_2_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m2_nofault = esc_2_m2_nofault[((esc_2_m2_nofault['TimeUS'] >= 163535671) & (esc_2_m2_nofault['TimeUS'] <= 244631223))]
esc_2_m2_nofault=esc_2_m2_nofault.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m2_nofault = pd.read_csv("ESC_3.csv")
esc_3_m2_nofault = esc_3_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m2_nofault = esc_3_m2_nofault[((esc_3_m2_nofault['TimeUS'] >= 163535671) & (esc_3_m2_nofault['TimeUS'] <= 244631223))]
esc_3_m2_nofault=esc_3_m2_nofault.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m2_nofault = pd.read_csv("ESC_4.csv")
esc_4_m2_nofault = esc_4_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m2_nofault = esc_4_m2_nofault[((esc_4_m2_nofault['TimeUS'] >= 163535671) & (esc_4_m2_nofault['TimeUS'] <= 244631223))]
esc_4_m2_nofault=esc_4_m2_nofault.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m2_nofault = pd.read_csv("ESC_5.csv")
esc_5_m2_nofault = esc_5_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m2_nofault = esc_5_m2_nofault[((esc_5_m2_nofault['TimeUS'] >= 163535671) & (esc_5_m2_nofault['TimeUS'] <= 244631223))]
esc_5_m2_nofault=esc_5_m2_nofault.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [8]:
#valori di accelerazione e del giroscopio
imu_0_m2_nofault = pd.read_csv("IMU_0.csv")
imu_0_m2_nofault = imu_0_m2_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m2_nofault = imu_0_m2_nofault[((imu_0_m2_nofault['TimeUS'] >= 163535671) & (imu_0_m2_nofault['TimeUS'] <= 244631223))]

imu_1_m2_nofault = pd.read_csv("IMU_1.csv")
imu_1_m2_nofault = imu_1_m2_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m2_nofault = imu_1_m2_nofault[((imu_1_m2_nofault['TimeUS'] >= 163535671) & (imu_1_m2_nofault['TimeUS'] <= 244631223))]

imu_2_m2_nofault = pd.read_csv("IMU_2.csv")
imu_2_m2_nofault = imu_2_m2_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m2_nofault = imu_2_m2_nofault[((imu_2_m2_nofault['TimeUS'] >= 163535671) & (imu_2_m2_nofault['TimeUS'] <= 244631223))]

imu_m2_nofault = pd.concat((imu_0_m2_nofault, imu_1_m2_nofault, imu_2_m2_nofault))
imu_m2_nofault=imu_m2_nofault.groupby(imu_m2_nofault.TimeUS, as_index=False).mean()

display(imu_m2_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,163537245,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200
1,163539870,0.125925,0.025622,0.027003,-0.082384,0.063078,-9.522074
2,163542660,0.030989,-0.038425,0.015617,-0.124611,-0.010491,-9.361492
3,163545177,-0.055869,-0.065410,0.012424,-0.162603,0.150488,-9.439348
4,163547584,0.044315,-0.006546,0.023296,-0.371309,0.336459,-9.679198
...,...,...,...,...,...,...,...
28162,244619831,0.128640,-0.060799,-0.040252,-0.261952,0.785947,-9.640874
28163,244622275,0.190746,-0.016192,-0.038782,-0.462853,0.624572,-9.764187
28164,244624838,0.057046,0.056984,-0.032290,-0.438465,0.580401,-9.704224
28165,244627437,0.037260,0.028643,-0.041810,-0.307790,0.681670,-9.576567


In [9]:
#sincronizzazione dei tempi tra att e imu_2
m2_nofault = pd.merge_ordered(imu_m2_nofault,att_m2_nofault)
m2_nofault=m2_nofault.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m2_nofault = pd.merge_ordered(m2_nofault, esc_0_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_1_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_2_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_3_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_4_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_5_m2_nofault)
m2_nofault=m2_nofault.fillna(method="ffill").fillna(method="bfill")
m2_nofault = pd.merge_ordered(m2_nofault, rcou_m2_nofault)
m2_nofault=m2_nofault.fillna(method="ffill").fillna(method="bfill")

In [10]:
index_to_remove=[]
for ind in att_m2_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m2_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m2_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m2_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m2_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m2_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m2_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m2_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

7391
8082
8765
9455
10147
10832
11506


In [11]:
m2_nofault = m2_nofault[~m2_nofault.TimeUS.isin(index_to_remove)]
m2_nofault["TimeUS"] = m2_nofault["TimeUS"] - m2_nofault.iloc[0]["TimeUS"]
m2_nofault["TimeUS"] = m2_nofault["TimeUS"].astype(int)
m2_nofault["Guasto"] = 0

In [12]:
from datetime import timedelta
m2_nofault=m2_nofault.reset_index(drop=True)
m2_nofault['TimeUS'] = pd.to_datetime(m2_nofault['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m2_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0
1,00:00:00.002625,0.125925,0.025622,0.027003,-0.082384,0.063078,-9.522074,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0
2,00:00:00.005415,0.030989,-0.038425,0.015617,-0.124611,-0.010491,-9.361492,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0
3,00:00:00.007932,-0.055869,-0.065410,0.012424,-0.162603,0.150488,-9.439348,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0
4,00:00:00.010339,0.044315,-0.006546,0.023296,-0.371309,0.336459,-9.679198,-0.13,1.17,-0.07,...,1.90,3454.0,1.83,1382.0,1426.0,1447.0,1359.0,1421.0,1387.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28162,00:01:21.082586,0.128640,-0.060799,-0.040252,-0.261952,0.785947,-9.640874,-2.42,-4.74,-0.87,...,2.05,5479.0,2.69,1577.0,1376.0,1489.0,1481.0,1476.0,1493.0,0
28163,00:01:21.085030,0.190746,-0.016192,-0.038782,-0.462853,0.624572,-9.764187,-2.34,-4.69,-0.86,...,2.05,5479.0,2.69,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0,0
28164,00:01:21.087593,0.057046,0.056984,-0.032290,-0.438465,0.580401,-9.704224,-2.34,-4.69,-0.86,...,2.05,5479.0,2.69,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0,0
28165,00:01:21.090192,0.037260,0.028643,-0.041810,-0.307790,0.681670,-9.576567,-2.34,-4.69,-0.86,...,2.05,5424.0,2.51,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0,0


## Analisi nel tempo e in frequenza

In [13]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [14]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m2_nofault)/350)):
        V1=m2_nofault[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [15]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = 0
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_23420\3012327637.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_23420\3012327637.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.140633,0.035760,-0.878642,0.235447,99.0,98.0,12.933143,5.857743,0.014846,...,5414.879212,3.303276,0.499700,-0.038840,3.377856,88.0,90.0,9.374849,8.836731,0
1,1.0,-0.140877,0.035679,-0.876481,0.235422,95.0,94.0,12.929592,5.842382,0.014615,...,5602.543034,3.305540,0.500080,-0.032475,3.380126,94.0,96.0,8.826495,8.682907,0
2,2.0,-0.141347,0.035656,-0.881007,0.235656,99.0,98.0,12.829186,5.721553,0.014690,...,5788.812412,3.307790,0.500447,-0.026112,3.382382,102.0,100.0,8.513056,8.251978,0
3,3.0,-0.141530,0.035566,-0.877453,0.235577,99.0,98.0,12.813405,5.677542,0.014600,...,5963.917246,3.310028,0.500803,-0.019750,3.384623,100.0,98.0,8.327253,7.657439,0
4,4.0,-0.141528,0.035466,-0.871380,0.235364,99.0,98.0,12.847111,5.675420,0.014353,...,6127.860290,3.312254,0.501146,-0.013388,3.386851,104.0,102.0,8.125583,7.048979,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75.0,-0.126804,0.032290,-0.643435,0.219758,0.0,128.0,26.984322,5.542090,-0.003981,...,5419.894236,3.453568,0.517608,0.396801,3.527538,101.0,100.0,8.866895,5.359051,0
76,76.0,-0.126395,0.032286,-0.642388,0.219513,0.0,126.0,27.413378,5.766863,-0.004472,...,5557.660061,3.455246,0.517595,0.402855,3.529179,95.0,94.0,9.372699,5.630240,0
77,77.0,-0.126124,0.032242,-0.638437,0.219257,0.0,132.0,27.780495,5.928746,-0.004583,...,5677.685092,3.456916,0.517576,0.408923,3.530812,103.0,102.0,9.841088,5.856800,0
78,78.0,-0.125752,0.032226,-0.636585,0.219007,0.0,128.0,28.188642,6.130657,-0.004753,...,5779.460426,3.458578,0.517552,0.415005,3.532437,105.0,104.0,10.269603,6.033221,0


In [17]:
path_file = path_file.replace(r"M2\NO_FAULT2", "")
os.chdir(path_file)
df_merged.to_csv('m2_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
